# Test Complex Queries over Multiple Documents (with and without Query Decomposition)

Query Decomposition: The ability to decompose a complex query into a simpler query given the content of the index.

Use OpenAI as the LLM model and embedding model.

In [1]:
import logging
import sys

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Uncomment if you want to temporarily disable logger
logger = logging.getLogger()
logger.disabled = True

In [2]:
from llama_index import (
    GPTVectorStoreIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext
)
import requests

/Users/suo/miniconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Load Datasets

Load Wikipedia pages as well as Paul Graham's "What I Worked On" essay

In [5]:
wiki_titles = ["Toronto", "Seattle", "San Francisco", "Chicago", "Boston", "Washington, D.C.", "Cambridge, Massachusetts", "Houston"]

In [13]:
from pathlib import Path
import requests

data_path = Path('data_wiki')

for title in wiki_titles:
    response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'format': 'json',
            'titles': title,
            'prop': 'extracts',
            # 'exintro': True,
            'explaintext': True,
        }
    ).json()
    page = next(iter(response['query']['pages'].values()))
    wiki_text = page['extract']

    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", 'w') as fp:
        fp.write(wiki_text)


In [14]:
# Load all wiki documents
city_docs = {}
all_docs = []
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(input_files=[data_path / f"{wiki_title}.txt"]).load_data()
    all_docs.extend(city_docs[wiki_title])


In [15]:
# define service context
service_context = ServiceContext.from_defaults(
    chunk_size=512, 
)

### Building the document indices
Build a separate vector index for each wiki pages about cities.

We also build a "global" vector index, which ingest documents for *all* cities. 

This allows us to test different types of data structures!

In [16]:
# Build index for each city document
city_indices = {}
index_summaries = {}
for wiki_title in wiki_titles:
    print(f"Building index for {wiki_title}")
    city_indices[wiki_title] = GPTVectorStoreIndex.from_documents(city_docs[wiki_title], service_context=service_context)
    # set summary text for city
    index_summaries[wiki_title] = f"Wikipedia articles about {wiki_title}"

Building index for Toronto


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 27294 tokens


Building index for Seattle


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 22263 tokens


Building index for San Francisco


...

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 28480 tokens


Building index for Cambridge, Massachusetts


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 17036 tokens


Building index for Houston


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 28795 tokens


In [17]:
# also setup a global vector index 
global_index = GPTVectorStoreIndex.from_documents(all_docs, service_context=service_context)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 213603 tokens


### Creating the right structure to run compare/contrast queries

Our key goal in this notebook is to run compare/contrast queries between different cities.

We currently have a separate vector index for every city document. We want to setup a "graph" structure in order to route the query 
in the right manner in order to retrieve the relevant text sections for each city. 

We compose a keyword table index on top of all the vector indices.

In [20]:
from llama_index.indices.composability import ComposableGraph

In [21]:
graph = ComposableGraph.from_indices(
    GPTSimpleKeywordTableIndex,
    [index for _, index in city_indices.items()], 
    [summary for _, summary in index_summaries.items()],
    max_keywords_per_chunk=50
)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens


### Define Query Transformation + Query Configs

We also define a "query decomposition" transform. Since we have a graph structure over multiple indexes, query decomposition
allows us to break a complex question into a simpler one over a given index.

This works well in comparing/contrasting different cities because it allows us to ask questions specific to each city.

**Query Transform**

In [24]:
from llama_index.indices.query.query_transform.base import DecomposeQueryTransform
decompose_transform = DecomposeQueryTransform(verbose=True)

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from llama_index.query_engine.transform_query_engine import TransformQueryEngine

custom_query_engines = {}
for index in city_indices.values():
    query_engine = index.as_query_engine(service_context=service_context)
    query_engine = TransformQueryEngine(
        query_engine,
        query_transform=decompose_transform,
        transform_extra_info={'index_summary': index.index_struct.summary},
    )
    custom_query_engines[index.index_id] = query_engine
custom_query_engines[graph.root_id] = graph.root_index.as_query_engine(
    retriever_mode='simple',
    response_mode='tree_summarize',
    service_context=service_context,
)


### Let's Run Some Queries! 

We run queries over the graphs and analyze the results.

We also compare results against the baseline global vector index. In the majority of cases the global vector index provides insufficient answers.

**Complex Query 1**

In [44]:
# with query decomposition in subindices
query_engine = graph.as_query_engine(custom_query_engines=custom_query_engines)
query_str = (
    "Compare and contrast the demographics in Seattle, Houston, and Toronto. "
)

In [45]:
response = query_engine.query(query_str)

INFO:llama_index.indices.keyword_table.retrievers:> Starting query: Compare and contrast the demographics in Seattle, Houston, and Toronto. 
INFO:llama_index.indices.keyword_table.retrievers:query keywords: ['demographics', 'seattle', 'toronto', 'compare', 'contrast', 'houston']
INFO:llama_index.indices.keyword_table.retrievers:> Extracted keywords: ['seattle', 'toronto', 'houston']


> Current query: Compare and contrast the demographics in Seattle, Houston, and Toronto. 
> New query:  What is the population of Seattle?


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 7 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1375 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1375 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 7 tokens


> Current query: Compare and contrast the demographics in Seattle, Houston, and Toronto. 
> New query:  What is the population of Toronto?


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1303 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1303 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


> Current query: Compare and contrast the demographics in Seattle, Houston, and Toronto. 
> New query:  What is the population of Houston?


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 7 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1401 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1401 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1681 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1681 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


In [46]:
print(str(response))



Seattle, Houston, and Toronto all have diverse populations, with immigrants making up a significant portion of the population in each city. However, the countries of origin for the immigrants vary between the cities. In Seattle, the top countries of origin for immigrants are Mexico, India, China, Philippines, and Vietnam. In Houston, the top countries of origin for immigrants are Mexico, India, El Salvador, Honduras, and Guatemala. In Toronto, the top countries of origin for immigrants are Philippines, China, India, Sri Lanka, and Jamaica. Additionally, the median age of the population varies between the cities. In Seattle, the median age is 37.2, in Houston it is 33.4, and in Toronto it is 39.2. Furthermore, the gender population also varies between the cities. In Seattle, the gender population is 48.2% male and 51.8% female, in Houston it is 48.3% male and 51.7% female, and in Toronto it is 48% male and 52% female. In 2016, the three most commonly reported ethnic origins overall we

In [47]:
query_engine = global_index.as_query_engine(
    similarity_top_k=3, 
    response_mode="tree_summarize"
)
response = query_engine.query(query_str)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 14 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 3549 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 3549 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


In [48]:
# NOTE: the global vector index seems to provide the right results....
# BUT see below! 
print(str(response))



Seattle is a major U.S. city located in the Pacific Northwest region of the United States. It has a population of over 730,000 people and is known for its high percentage of college and university graduates. Of the city's population over the age of 25, 53.8% hold a bachelor's degree or higher, and 91.9% have a high school diploma or equivalent. Seattle is also home to the University of Washington, as well as a number of smaller private universities such as Seattle Pacific University, a Jesuit Catholic institution, and Seattle University, a Free Methodist institution. The Seattle Colleges District operates three colleges: North Seattle College, Seattle Central College, and South Seattle College. According to a 2006 study by UCLA, 12.9% of city residents polled identified as gay, lesbian, or bisexual. This was the second-highest proportion of any major U.S. city, behind San Francisco. Seattle's economy is driven by a mix of older industrial companies and "new economy" internet and tech

In [49]:
# NOTE: there's hallucination! the sources only reference Toronto
print(response.source_nodes[0].source_text)
print(response.source_nodes[1].source_text)

Tiffany Washington, and Kendee Yamaguchi.


== Education ==

Of the city's population over the age of 25, 53.8% (vs. a national average of 27.4%) hold a bachelor's degree or higher, and 91.9% (vs. 84.5% nationally) have a high school diploma or equivalent. A 2008 United States Census Bureau survey showed that Seattle had the highest percentage of college and university graduates of any major U.S. city. The city was listed as the most literate of the country's 69 largest cities in 2005 and 2006, the second most literate in 2007 and the most literate in 2008 in studies conducted by Central Connecticut State University.Seattle Public Schools is the school district for the vast majority of the city. That school district desegregated without a court order but continue to struggle to achieve racial balance in a somewhat ethnically divided city (the south part of town having more ethnic minorities than the north). In 2007, Seattle's racial tie-breaking system was struck down by the United Sta

/Users/suo/dev/llama_index/llama_index/data_structs/node.py:176: UserWarning: .source_text is deprecated, use .node.get_text() instead
  warnings.warn(".source_text is deprecated, use .node.get_text() instead")


**Complex Query 2**

In [ ]:
# with query decomposition
query_str = (
    "What are the basketball teams in Houston and Boston?"
)

In [ ]:
query_engine = graph.as_query_engine(custom_query_engines=custom_query_engines)

response = query_engine.query(query_str)

In [ ]:
print(str(response))

In [ ]:
query_engine = global_index.as_query_engine(
    similarity_top_k=2, 
    response_mode="tree_summarize"
)
response = query_engine.query(query_str)

In [ ]:
print(str(response))

**Complex Query 3**

In [ ]:
# with query decomposition
query_str = (
    "Compare and contrast the climate of Houston and Boston "
)

In [ ]:
query_engine = graph.as_query_engine(custom_query_engines=custom_query_engines)

response = query_engine.query(query_str)

In [ ]:
print(response)

In [ ]:
query_engine = global_index.as_query_engine(
    similarity_top_k=2, 
    response_mode="tree_summarize"
)
response = query_engine.query(query_str)

In [ ]:
print(str(response))